In [360]:
# Dữ liệu sử dụng là một tập dữ liệu về thẻ tín dụng, bao gồm thông tin cá nhân và tài chính của khách hàng.
#Các thuộc tính bao gồm:
#ID khách hàng, giới tính, sở hữu ô tô và bất động sản,số lượng con cái, thu nhập, loại thu nhập,
# trình độ học vấn, tình trạng hôn nhân,cách thức sống, ngày sinh, ngày bắt đầu làm việc,
#có sở hữu điện thoại và email không,  nghề nghiệp, và số lượng thành viên trong gia đình.


# Dữ liệu đã được phân tích rõ trong bản báo cáo.

# Tên thành viên
#   1 NGUYỄN THỊ THU YẾN - 52000869
#   2 NGUYỄN KHẮC VĂN - 52000868
#   3 TRƯƠNG THÔNG THẾ THÁI - 52000714


# Import các thư viện cần thiết

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'


import sys
ENV_COLAB = 'google.colab' in sys.modules

if ENV_COLAB:
    print('Environment: Google Colab')
    !pip install catboost
    !pip install ipywidgets
    !jupyter nbextension enable --py widgetsnbextension

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding, SimpleRNN
from tensorflow.keras.layers import Dense, Dropout


from imblearn.over_sampling import SMOTE
import itertools
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier



from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

Environment: Google Colab


# 1.1 Đọc dữ liệu, thay đổi tên cột để chuẩn bị dữ liệu làm việc

In [ ]:
# Đọc data
data = pd.read_csv("credit-card-approval-prediction/application_record.csv")
record = pd.read_csv("credit-card-approval-prediction/credit_record.csv")

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
# Tìm tất cả các tài khoản người dùng mở trong tháng
# nhóm DataFrame theo cột "ID" và tính giá trị tối thiểu của cột "MONTHS_BALANCE" cho mỗi nhóm
begin_month=pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min))
# Đổi tên cột "MONTHS_BALANCE" trong DataFrame đầu tháng thành "begin_month1".
begin_month=begin_month.rename(columns={'MONTHS_BALANCE':'begin_month1'})
new_data=pd.merge(data,begin_month,how="left",on="ID") #merge to record data

Tạo một cột mới có tên là "dep_value" trong DataFrame ghi dữ liệu dựa trên giá trị trong cột "STATUS". Đặt "Yes" trong cột "dep_value" cho các hàng mà cột "STATUS" có giá trị '2', '3', '4' hoặc '5'.

In [ ]:
record['dep_value'] = None
record['dep_value'][record['STATUS'] =='2']='Yes'
record['dep_value'][record['STATUS'] =='3']='Yes'
record['dep_value'][record['STATUS'] =='4']='Yes'
record['dep_value'][record['STATUS'] =='5']='Yes'

Tạo một cột mới 'dep_value' trong DataFrame "cpunt" dựa trên số lần xuất hiện của 'dep_value' trong DataFrame "record". Sau đó, hợp nhất thông tin này vào DataFrame "new_data" dựa trên cột "ID" và tạo một cột mới 'target' dựa trên cột 'dep_value'.



In [ ]:
cpunt=record.groupby('ID').count()
cpunt['dep_value'][cpunt['dep_value'] > 0]='Yes'
cpunt['dep_value'][cpunt['dep_value'] == 0]='No'
cpunt = cpunt[['dep_value']]
new_data=pd.merge(new_data,cpunt,how='inner',on='ID')
new_data['target']=new_data['dep_value']
new_data.loc[new_data['target']=='Yes','target']=1
new_data.loc[new_data['target']=='No','target']=0

In ấn số lượng giá trị duy nhất trong cột 'dep_value' của DataFrame "cpunt" và sau đó in số lượng giá trị chuẩn hóa.

In [ ]:
print(cpunt['dep_value'].value_counts())
print(cpunt['dep_value'].value_counts(normalize=True))

Đổi tên các cột trong DataFrame "new_data" bằng cách sử dụng phương pháp rename.

In [ ]:
new_data.rename(columns={'CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car','FLAG_OWN_REALTY':'Reality',
                         'CNT_CHILDREN':'ChldNo','AMT_INCOME_TOTAL':'inc',
                         'NAME_EDUCATION_TYPE':'edutp','NAME_FAMILY_STATUS':'famtp',
                        'NAME_HOUSING_TYPE':'houtp','FLAG_EMAIL':'email',
                         'NAME_INCOME_TYPE':'inctp','FLAG_WORK_PHONE':'wkphone',
                         'FLAG_PHONE':'phone','CNT_FAM_MEMBERS':'famsize',
                        'OCCUPATION_TYPE':'occyp'
                        },inplace=True)
# Xử lý các giá trị bị thiếu trong new_data DataFrame bằng phương thức dropna.
new_data.dropna()
new_data = new_data.mask(new_data == 'NULL').dropna()

Tạo một bảng giá trị (IV) cho các biến trong DataFrame "new_data", sau đó loại bỏ một số biến được chỉ định trong danh sách tên.

Tại đây khởi tạo một bảng IV với tên biến trong cột biến và một cột giữ chỗ 'IV'. Sau đó, nó lặp qua danh sách tên và loại bỏ các hàng tương ứng với các biến được chỉ định từ bảng IV.

In [ ]:
ivtable=pd.DataFrame(new_data.columns,columns=['variable'])
ivtable['IV']=None
namelist = ['FLAG_MOBIL','begin_month','dep_value','target','ID']

for i in namelist:
    ivtable.drop(ivtable[ivtable['variable'] == i].index, inplace=True)
ivtable


# 1.2 Tạo một số hàm hỗ trợ


Định nghĩa một hàm calc_iv để tính Giá trị(IV) cho một đặc trưng cụ

*   List item
*   List item

thể trong DataFrame. Hàm này tính toán các thước đo khác nhau liên quan đến phân phối của đặc trưng đối với biến mục tiêu.

In [ ]:
def calc_iv(df, feature, target, pr=False):
    lst = []
    df[feature] = df[feature].fillna("NULL")

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature,                                                        # Variable
                    val,                                                            # Value
                    df[df[feature] == val].count()[feature],                        # All
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # Good (think: Fraud == 0)
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]]) # Bad (think: Fraud == 1)

    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])
    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])

    data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
    data.index = range(len(data.index))

    if pr:
        print(data)
        print('IV = ', data['IV'].sum())

    iv = data['IV'].sum()
    print('This variable\'s IV is:',iv)
    print(df[feature].value_counts())
    return iv, data

Định nghĩa một hàm convert_dummy để chuyển đổi một đặc trưng phân loại thành biến giả bằng cách sử dụng mã hóa one-hot.

In [ ]:
def convert_dummy(df, feature,rank=0):
    pos = pd.get_dummies(df[feature], prefix=feature)
    mode = df[feature].value_counts().index[rank]
    biggest = feature + '_' + str(mode)
    pos.drop([biggest],axis=1,inplace=True)
    df.drop([feature],axis=1,inplace=True)
    df=df.join(pos)
    return df

Định nghĩa một hàm get_category để phân loại một cột số trong DataFrame thành các khoảng rời rạc. Hàm này sử dụng enther pd.qcut (cắt theo tỷ lệ) hoặc pd.cut (cắt theo chiều dài bằng nhau) dựa trên giá trị của tham số qcut

In [ ]:
def get_category(df, col, binsnum, labels, qcut = False):
    if qcut:
        localdf = pd.qcut(df[col], q = binsnum, labels = labels) # quantile cut
    else:
        localdf = pd.cut(df[col], bins = binsnum, labels = labels) # equal-length cut

    localdf = pd.DataFrame(localdf)
    name = 'gp' + '_' + col
    localdf[name] = localdf[col]
    df = df.join(localdf[name])
    df[name] = df[name].astype(object)
    return df


Định nghĩa một hàm có tên là plot_confusion_matrix để trực quan hóa ma trận. Hàm này sử dụng Matplotlib để tạo một biểu đồ heatmap biểu diễn ma trận.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# 1.3 Phân tích dữ liệu, tiền xử lý dữ liệu

Chuyển đổi cột 'Gender' trong DataFrame "new_data" từ giá trị phân loại ('F' và 'M') thành giá trị số học (0 và 1), sau đó tính Giá trị(IV) cho biến đã được biến đổi này.

In [ ]:
new_data['Gender'] = new_data['Gender'].replace(['F','M'],[0,1])
print(new_data['Gender'].value_counts())
iv, data = calc_iv(new_data,'Gender','target')
ivtable.loc[ivtable['variable']=='Gender','IV']=iv
data.head()

Thay thế giá trị 'N' và 'Y' trong cột 'Car' bằng 0 và 1 tương ứng.

In [ ]:
new_data['Car'] = new_data['Car'].replace(['N','Y'],[0,1])
print(new_data['Car'].value_counts())
iv, data=calc_iv(new_data,'Car','target')
ivtable.loc[ivtable['variable']=='Car','IV']=iv
data.head()

Thay thế 'N' bằng 0 và 'Y' bằng 1 trong cột 'Reality', in số lượng giá trị, tính IV bằng cách sử dụng hàm calc_iv, cập nhật IV trong DataFrame "ivtable".

In [ ]:
new_data['Reality'] = new_data['Reality'].replace(['N','Y'],[0,1])
print(new_data['Reality'].value_counts())
iv, data=calc_iv(new_data,'Reality','target')
ivtable.loc[ivtable['variable']=='Reality','IV']=iv
data.head()

Chuyển đổi cột 'phone' sang kiểu chuỗi, sau đó in giá trị chuẩn hóa, loại bỏ các hàng mà 'phone' là 'nan', và cuối cùng tính toán Giá trị(IV) bằng cách sử dụng hàm calc_iv.

In [ ]:
new_data['phone']=new_data['phone'].astype(str)
print(new_data['phone'].value_counts(normalize=True,sort=False))
new_data.drop(new_data[new_data['phone'] == 'nan' ].index, inplace=True)
iv, data=calc_iv(new_data,'phone','target')
ivtable.loc[ivtable['variable']=='phone','IV']=iv
data.head()

In giá trị chuẩn hóa, sau đó chuyển đổi cột 'email' sang kiểu chuỗi, và cuối cùng tính Giá trị (IV) bằng cách sử dụng hàm calc_iv.

In [ ]:
print(new_data['email'].value_counts(normalize=True,sort=False))
new_data['email']=new_data['email'].astype(str)
iv, data=calc_iv(new_data,'email','target')
ivtable.loc[ivtable['variable']=='email','IV']=iv
data.head()

Xử lý cột 'wkphone' theo cách tương tự như mã nguồn trước. Đầu tiên, chuyển đổi 'wkphone' sang kiểu chuỗi, tính toán Giá trị(IV) bằng cách sử dụng hàm calc_iv, loại bỏ các hàng mà 'wkphone' là 'nan', và sau đó cập nhật IV trong DataFrame "ivtable".

In [ ]:
new_data['wkphone']=new_data['wkphone'].astype(str)
iv, data = calc_iv(new_data,'wkphone','target')
new_data.drop(new_data[new_data['wkphone'] == 'nan' ].index, inplace=True)
ivtable.loc[ivtable['variable']=='wkphone','IV']=iv
data.head()

Phân loại cột 'ChldNo' trong DataFrame "new_data" của bạn. Nếu giá trị trong cột 'ChldNo' lớn hơn hoặc bằng 2, bạn sẽ thay thế nó bằng '2More'.

In [ ]:
new_data.loc[new_data['ChldNo'] >= 2,'ChldNo']='2More'
print(new_data['ChldNo'].value_counts(sort=False))

Tính Giá trị (IV) cho cột 'ChldNo' trong DataFrame "new_data" sau khi phân loại nó thành các nhóm khác nhau.

In [ ]:
iv, data=calc_iv(new_data,'ChldNo','target')
ivtable.loc[ivtable['variable']=='ChldNo','IV']=iv
data.head()

In [ ]:
new_data = convert_dummy(new_data,'ChldNo')

Thực hiện một số xử lý tiền xử lý trên cột 'inc' (thu nhập) trong DataFrame "new_data". Cụ thể, chuyển đổi cột 'inc' thành kiểu đối tượng, và chia giá trị cho 10,000

In [ ]:
new_data['inc']=new_data['inc'].astype(object)
new_data['inc'] = new_data['inc']/10000
print(new_data['inc'].value_counts(bins=10,sort=False))
new_data['inc'].plot(kind='hist',bins=50,density=True)

Tính toán tuổi từ cột 'DAYS_BIRTH' trong DataFrame "new_data", sau đó phân tích phân phối tuổi bằng cách sử dụng hàm đếm giá trị (value counts) và một biểu đồ histogram.

In [ ]:
new_data['Age']=-(new_data['DAYS_BIRTH'])//365
print(new_data['Age'].value_counts(bins=10,normalize=True,sort=False))
new_data['Age'].plot(kind='hist',bins=20,density=True)

Phân loại cột 'Age' thành năm nhóm bằng cách sử dụng hàm get_category, sau đó tính Giá trị (IV) cho biến nhóm mới ('gp_Age').

In [ ]:
new_data = get_category(new_data,'Age',5, ["lowest","low","medium","high","highest"])
iv, data = calc_iv(new_data,'gp_Age','target')
ivtable.loc[ivtable['variable']=='DAYS_BIRTH','IV'] = iv
data.head()

In [ ]:
new_data = convert_dummy(new_data,'gp_Age')

Tính số năm làm việc ('worktm') dựa trên cột 'DAYS_EMPLOYED' trong DataFrame "new_data". Sau khi tính toán thời gian làm việc, xử lý các giá trị âm (có thể chỉ ra dữ liệu thiếu hoặc dữ liệu ngoại lệ) bằng cách thay thế chúng bằng NaN, và sau đó điền giá trị NaN bằng giá trị trung bình của cột 'worktm'.

In [ ]:
new_data['worktm']=-(new_data['DAYS_EMPLOYED'])//365
new_data[new_data['worktm']<0] = np.nan # replace by na
new_data['DAYS_EMPLOYED']
new_data['worktm'].fillna(new_data['worktm'].mean(),inplace=True) #replace na by mean
new_data['worktm'].plot(kind='hist',bins=20,density=True)

Phân loại cột 'worktm' (số năm làm việc) thành năm nhóm bằng cách sử dụng hàm get_category, sau đó tính Giá trị (IV) cho biến nhóm mới ('gp_worktm').

In [ ]:
new_data = get_category(new_data,'worktm',5, ["lowest","low","medium","high","highest"])
iv, data=calc_iv(new_data,'gp_worktm','target')
ivtable.loc[ivtable['variable']=='DAYS_EMPLOYED','IV']=iv
data.head()

In [ ]:
new_data = convert_dummy(new_data,'gp_worktm')
new_data['famsize'].value_counts(sort=False)

Thao tác trên cột 'famsize' trong DataFrame "new_data". Sau đó, chuyển đổi nó thành kiểu số nguyên, tạo một cột mới 'famsizegp', phân loại nó, và sau đó tính Giá trị(IV) cho biến đã phân loại.

In [ ]:
new_data['famsize']=new_data['famsize'].astype(int)
new_data['famsizegp']=new_data['famsize']
new_data['famsizegp']=new_data['famsizegp'].astype(object)
new_data.loc[new_data['famsizegp']>=3,'famsizegp']='3more'
iv, data=calc_iv(new_data,'famsizegp','target')
ivtable.loc[ivtable['variable']=='famsize','IV']=iv
data.head()

In [ ]:
new_data = convert_dummy(new_data,'famsizegp')


Chỉnh sửa giá trị của cột 'inctp' bằng cách kết hợp các hạng mục 'Pensioner' và 'Student' thành 'State servant'. Sau các sửa đổi này, tính Giá trị (IV) cho biến 'inctp'.

In [ ]:
print(new_data['inctp'].value_counts(sort=False))
print(new_data['inctp'].value_counts(normalize=True,sort=False))
new_data.loc[new_data['inctp']=='Pensioner','inctp']='State servant'
new_data.loc[new_data['inctp']=='Student','inctp']='State servant'
iv, data=calc_iv(new_data,'inctp','target')
ivtable.loc[ivtable['variable']=='inctp','IV']=iv
data.head()

In [ ]:
new_data = convert_dummy(new_data,'inctp')

Phân loại lại cột 'occyp' (loại nghề nghiệp) trong DataFrame "new_data" bằng cách nhóm một số nghề nghiệp cụ thể vào các danh mục rộng hơn. Sau quá trình phân loại lại này, tính Giá trị (IV) cho biến 'occyp' đã được sửa đổi

In [ ]:
new_data.loc[(new_data['occyp']=='Cleaning staff') | (new_data['occyp']=='Cooking staff') | (new_data['occyp']=='Drivers') | (new_data['occyp']=='Laborers') | (new_data['occyp']=='Low-skill Laborers') | (new_data['occyp']=='Security staff') | (new_data['occyp']=='Waiters/barmen staff'),'occyp']='Laborwk'
new_data.loc[(new_data['occyp']=='Accountants') | (new_data['occyp']=='Core staff') | (new_data['occyp']=='HR staff') | (new_data['occyp']=='Medicine staff') | (new_data['occyp']=='Private service staff') | (new_data['occyp']=='Realty agents') | (new_data['occyp']=='Sales staff') | (new_data['occyp']=='Secretaries'),'occyp']='officewk'
new_data.loc[(new_data['occyp']=='Managers') | (new_data['occyp']=='High skill tech staff') | (new_data['occyp']=='IT staff'),'occyp']='hightecwk'
print(new_data['occyp'].value_counts())
iv, data=calc_iv(new_data,'occyp','target')
ivtable.loc[ivtable['variable']=='occyp','IV']=iv
data.head()

In [ ]:
new_data = convert_dummy(new_data,'occyp')

Tính Giá trị (IV) cho cột 'houtp' (loại nhà ở) trong DataFrame "new_data".

In [ ]:
iv, data=calc_iv(new_data,'houtp','target')
ivtable.loc[ivtable['variable']=='houtp','IV']=iv
data.head()

In [ ]:
new_data = convert_dummy(new_data,'houtp')

Chỉnh sửa cột 'edutp' (loại học vấn) trong DataFrame "new_data" bằng cách thay thế hạng mục 'Academic degree' bằng 'Higher education'. Sau khi sửa đổi này, tính Giá trị (IV) cho biến 'edutp' đã được sửa đổi.

In [ ]:
new_data.loc[new_data['edutp']=='Academic degree','edutp']='Higher education'
iv, data=calc_iv(new_data,'edutp','target')
ivtable.loc[ivtable['variable']=='edutp','IV']=iv
data.head()

In [ ]:
new_data = convert_dummy(new_data,'edutp')

In [ ]:
new_data['famtp'].value_counts(normalize=True,sort=False)

In [ ]:
new_data['famtp'].value_counts(normalize=True,sort=False)

Tính Giá trị (IV) cho cột 'famtp' (tình trạng gia đình) trong DataFrame "new_data".

In [ ]:
iv, data=calc_iv(new_data,'famtp','target')
ivtable.loc[ivtable['variable']=='famtp','IV']=iv
data.head()

In [ ]:
new_data = convert_dummy(new_data,'famtp')

Sắp xếp DataFrame "ivtable" theo Giá trị (IV) giảm dần, sau đó chỉnh sửa tên biến để đọc dễ hiểu hơn.

In [ ]:
ivtable=ivtable.sort_values(by='IV',ascending=False)
ivtable.loc[ivtable['variable']=='DAYS_BIRTH','variable']='agegp'
ivtable.loc[ivtable['variable']=='DAYS_EMPLOYED','variable']='worktmgp'
ivtable.loc[ivtable['variable']=='inc','variable']='incgp'
ivtable

Định nghĩa biến mục tiêu Y và ma trận đặc trưng X cho một nhiệm vụ mô hình hóa dự đoán. Ở đây, Y đại diện cho biến mục tiêu ('target'), và X bao gồm một lựa chọn các đặc trưng từ DataFrame "new_data" của bạn

In [ ]:
Y = new_data['target']
X = new_data[['ID','Gender','Reality','ChldNo_1', 'ChldNo_2More','wkphone',
              'gp_Age_high', 'gp_Age_highest', 'gp_Age_low',
       'gp_Age_lowest','gp_worktm_high', 'gp_worktm_highest',
       'gp_worktm_low', 'gp_worktm_medium','occyp_hightecwk',
              'occyp_officewk','famsizegp_1', 'famsizegp_3more',
       'houtp_Co-op apartment', 'houtp_Municipal apartment',
       'houtp_Office apartment', 'houtp_Rented apartment',
       'houtp_With parents','edutp_Higher education',
       'edutp_Incomplete higher', 'edutp_Lower secondary','famtp_Civil marriage',
       'famtp_Separated','famtp_Single / not married','famtp_Widow']]

Thực hiện tái chọn mẫu sử dụng Phương pháp Tổng hợp Thấp hơn Thiểu số (SMOTE) để cân bằng phân phối lớp trong biến mục tiêu Y của bạn.

In [ ]:
Y = Y.astype('int')
X_balance,Y_balance = SMOTE().fit_resample(X,Y)
X_balance = pd.DataFrame(X_balance, columns = X.columns)

# 2.1 Giải quyết bài toán bằng các mô hình học máy

Chia dữ liệu đã được tái chọn mẫu thành các tập huấn luyện và kiểm thử bằng cách sử dụng hàm train_test_split.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_balance,Y_balance,
                                                    stratify=Y_balance, test_size=0.3,
                                                    random_state = 10086)

In [ ]:
X_train.head()

Trích xuất cột 'ID' từ tập huấn luyện và testing

In [ ]:
X_train_ID = X_train['ID']
X_test_ID = X_test['ID']

In [ ]:
X_test_ID = X_test['ID']


In [ ]:
X_test.head()

training logistic regression model

In [ ]:
#Logistic Regression
model = LogisticRegression(C=0.8,
                           random_state=0,
                           solver='lbfgs')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

sns.set_style('white')
class_names = ['0','1']
plot_confusion_matrix(confusion_matrix(y_test,y_predict),
                      classes= class_names,
                      title='Confusion Matrix: Logistic Regression')

Kiểm tra độ dài của các nhãn dự đoán (y_predict), nhãn thực tế (y_test), và cột 'ID' trong tập kiểm thử (X_test_ID).

In [ ]:
len(y_predict),len(y_test),len(X_test_ID)

Tạo một DataFrame mới, final_prediction_comparison, để so sánh các nhãn dự đoán (y_predict) với nhãn thực tế (y_test) cùng với các giá trị 'ID' tương ứng từ tập kiểm thử (X_test_ID).

In [ ]:
data_dict = {'ID':X_test_ID,'y_test':y_test,'y_predict':y_predict}
final_prediction_caparison = pd.DataFrame(data_dict)
final_prediction_caparison

Decision tree model

In [ ]:
#decision tree
model = DecisionTreeClassifier(max_depth=12,
                               min_samples_split=8,
                               random_state=1024)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

dt_accuracy_before = accuracy_score(y_test, y_predict)
print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

plot_confusion_matrix(confusion_matrix(y_test,y_predict),
                      classes=class_names,
                      title='Confusion Matrix: Decision Tree')

Random forest

In [ ]:
# Random forest
model = RandomForestClassifier(n_estimators=250,
                              max_depth=12,
                              min_samples_leaf=16
                              )
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

rf_accuracy_before = accuracy_score(y_test, y_predict)
print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

plot_confusion_matrix(confusion_matrix(y_test,y_predict),
                      classes=class_names,
                      title='Confusion Matrix: Random Forests')

In [ ]:
# Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)


gb_model.fit(X_train, y_train)
y_gb_predict = gb_model.predict(X_test)

gb_accuracy_before = accuracy_score(y_test, y_predict)
print('Gradient Boosting Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_gb_predict)))
print(pd.DataFrame(confusion_matrix(y_test, y_gb_predict)))

plot_confusion_matrix(confusion_matrix(y_test, y_gb_predict),
                      classes=class_names,
                      title='Confusion Matrix: Gradient Boosting')


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# K-Nearest Neighbors (KNN)
knn_model = KNeighborsClassifier(n_neighbors=5)

knn_model.fit(X_train, y_train)
y_knn_predict = knn_model.predict(X_test)

knn_accuracy_before = accuracy_score(y_test, y_knn_predict)
print(knn_accuracy_before)
print('KNN Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_knn_predict)))
print(pd.DataFrame(confusion_matrix(y_test, y_knn_predict)))


plot_confusion_matrix(confusion_matrix(y_test, y_knn_predict),
                      classes=class_names,
                      title='Confusion Matrix: K-Nearest Neighbors')

# 3.1 Sử dụng Feed Forward Neural Network và Reccurent Neural Network để giải quyết bài toán

In [ ]:
# Chuyển đổi dữ liệu đầu vào sang kiểu dữ liệu float32
X_train_FNN = np.asarray(X_train).astype(np.float32)
X_test_FNN = np.asarray(X_test).astype(np.float32)
y_train_FNN = np.asarray(y_train).astype(np.float32)
y_test_FNN = np.asarray(y_test).astype(np.float32)

fnn_model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

fnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


fnn = fnn_model.fit(X_train_FNN, y_train_FNN, epochs=10, batch_size=32, validation_data=(X_test_FNN, y_test_FNN))
fnn_loss, fnn_accuracy = fnn_model.evaluate(X_test_FNN, y_test_FNN)

print(f'FNN Accuracy Score is: {fnn_accuracy:.4f}')

In [ ]:
X_train_3D = X_train_FNN.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_3D = X_test_FNN.reshape((X_test_FNN.shape[0], 1, X_test_FNN.shape[1]))
y_train_RNN = np.asarray(y_train_FNN).astype(np.float32)
y_test_RNN = np.asarray(y_test_FNN).astype(np.float32)
# Tạo mô hình RNN
rnn_model = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(1, X_train_FNN.shape[1])),
    Dense(1, activation='sigmoid')
])

rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn = rnn_model.fit(X_train_3D, y_train_RNN, epochs=10, batch_size=32, validation_data=(X_test_3D, y_test_RNN))

rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test_3D, y_test_RNN)
print(f'RNN Accuracy Score is: {rnn_accuracy:.4f}')

# 4.1 Áp dụng các kỹ thuật tránh Overfiting các mô hình học máy

In [ ]:
model = DecisionTreeClassifier(
    max_depth=15,
    min_samples_split=12,
    min_samples_leaf=12,
    random_state=1024
)

model.fit(X_train, y_train)
y_predict = model.predict(X_test)


dt_accuracy_after = accuracy_score(y_test, y_predict)
print(' Decision Tree Accuracy Score is {:.5}'.format(dt_accuracy_after))
print(pd.DataFrame(confusion_matrix(y_test, y_predict)))

plot_confusion_matrix(confusion_matrix(y_test, y_predict),
                      classes=class_names,
                      title='Confusion Matrix: Decision Tree')

print('\n Decision Tree Accuracy Score before is ', dt_accuracy_before)
print(' Decision Tree Accuracy Score after is ', dt_accuracy_after)

In [ ]:
# Adjusted parameters to prevent overfitting
gb_model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.05, max_depth=5, min_samples_split=5,
                                      min_samples_leaf=2, random_state=42)

gb_model.fit(X_train, y_train)
y_gb_predict = gb_model.predict(X_test)

gb_accuracy_after = accuracy_score(y_test, y_gb_predict)
print('Gradient Boosting Accuracy Score is {:.5}'.format(gb_accuracy_after))
print(pd.DataFrame(confusion_matrix(y_test, y_gb_predict)))

plot_confusion_matrix(confusion_matrix(y_test, y_gb_predict),
                      classes=class_names,
                      title='Confusion Matrix: Gradient Boosting')

print('\nGradient Boosting Accuracy Score before is ', gb_accuracy_before)
print('Gradient Boosting Accuracy Score after is ', gb_accuracy_after)

In [ ]:
#random forest
model = RandomForestClassifier(n_estimators=350,
                              max_depth=15,
                              min_samples_leaf=20
                              )
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

rf_accuracy_after = accuracy_score(y_test, y_predict)
print('Random forest Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

plot_confusion_matrix(confusion_matrix(y_test,y_predict),
                      classes=class_names,
                      title='Confusion Matrix: Random Forests')


print('\nRandom forest Accuracy Score before is ', rf_accuracy_before)
print('Random forest Accuracy Score after is ', rf_accuracy_after)


In [ ]:
# K-Nearest Neighbors (KNN)
knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform')  # Điều chỉnh số láng giềng và trọng số nếu cần thiết
knn_model.fit(X_train, y_train)

y_knn_predict = knn_model.predict(X_test)
knn_accuracy_after = accuracy_score(y_test, y_knn_predict)
print('KNN Accuracy Score is {:.5}'.format(knn_accuracy_after))
print(pd.DataFrame(confusion_matrix(y_test, y_knn_predict)))

plot_confusion_matrix(confusion_matrix(y_test, y_knn_predict),
                      classes=class_names,
                      title='Confusion Matrix: K-Nearest Neighbors')

print('\nKNN Accuracy Score before is ', knn_accuracy_before)
print('KNN Accuracy Score after is ', knn_accuracy_after)

# 4.2 Áp dụng các kỹ thuật tránh Overfiting cho FNN và RNN

In [ ]:
def plot_comparison(history1, history2, metric='accuracy'):
    plt.plot(history1.history[metric], label='Without Overfitting')
    plt.plot(history2.history[metric], label='With Overfitting')
    plt.title(f'Model Comparison - {metric}')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
    plt.show()


In [ ]:
# Tạo mô hình FNN có sử dụng tránh Overfiting
fnn_model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

fnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
fnn_dropout = fnn_model.fit(X_train_FNN, y_train_FNN, epochs=10, batch_size=32, validation_data=(X_test_FNN, y_test_FNN))


fnn_loss, fnn_accuracy_dropout = fnn_model.evaluate(X_test_FNN, y_test_FNN)
print(f'FNN Accuracy Score is: {fnn_accuracy:.4f}')

In [ ]:
# Tạo mô hình RNN có sử dụng tránh Overfiting
rnn_model = Sequential([
    Dropout(0.5),
    SimpleRNN(50, activation='relu', input_shape=(1, X_train_FNN.shape[1])),
    Dense(1, activation='sigmoid')
])

rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


rnn_dropout = rnn_model.fit(X_train_3D, y_train_RNN, epochs=10, batch_size=32, validation_data=(X_test_3D, y_test_RNN))

rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test_3D, y_test_RNN)
print(f'RNN Accuracy Score is: {rnn_accuracy:.4f}')

In [ ]:
# So sánh độ chính xác
plot_comparison(fnn, fnn_dropout, metric='accuracy')
# So sánh hàm mất mát
plot_comparison(fnn, fnn_dropout, metric='loss')

In [ ]:
# So sánh độ chính xác
plot_comparison(rnn, rnn_dropout, metric='accuracy')
# So sánh hàm mất mát
plot_comparison(rnn, rnn_dropout, metric='loss')


# 5.1 Các giải pháp để cải tiến mô hình

Để cải thiện độ chính xác của mô hình sau khi đã huấn luyện, chúng ta có thể thực hiện các bước sau:

1.   Phân tích Trường hợp Sai: Xem xét các trường hợp mà mô hình dự đoán sai. Phân tích các false positives và false negatives để hiểu rõ hơn về lý do mô hình đưa ra các quyết định không chính xác.

2.   Tinh chỉnh Tham số Mô hình: Kiểm tra và điều chỉnh các tham số của mô hình. Sử dụng kỹ thuật tinh chỉnh hyperparameters như Grid Search hoặc Random Search để tìm ra các giá trị tham số tốt nhất.

3. Tăng cường Dữ liệu (Data Augmentation): Tăng kích thước của tập dữ liệu bằng cách thêm các biến thể của các mẫu hiện có hoặc tạo dữ liệu tổng hợp. Điều này có thể giúp mô hình học được từ nhiều biến thể hơn và cải thiện khả năng tổng quát hóa.

4. Điều chỉnh các yếu tố như độ sâu của cây (đối với cây quyết định), số lượng cây (đối với rừng ngẫu nhiên), hay số lượng layer và units (đối với mạng nơ-ron) để kiểm soát overfitting và underfitting.

Biện pháp đã sử dụng trong bài toán:

1.   Tinh chỉnh Tham số Mô hình: Kiểm tra và điều chỉnh các tham số của mô hình. Sử dụng kỹ thuật tinh chỉnh hyperparameters như Grid Search hoặc Random Search để tìm ra các giá trị tham số tốt nhất.

2. Điều chỉnh các yếu tố như độ sâu của cây (đối với cây quyết định), số lượng cây (đối với rừng ngẫu nhiên), hay số lượng layer và units (đối với mạng nơ-ron) để kiểm soát overfitting và underfitting.

**==> Kết quả đã được cải thiện, mô hình học chính xác hơn:**

